In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import nibabel as nib
import os
import traceback
import joblib

In [3]:
from elastix import *
from laplacian3DRegistration import *
from reg_utils import *
from reconstruction import *
from plot_utils import *

from registration.utils import loadNiiImages, create_nifti_image, getMutualInformation

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
from skimage.color import gray2rgb, label2rgb
import skimage
def convertToUint8(data):
    """
    Thresholds  and converts data to UINT8
    """
    maxVal  = np.percentile(data, 99)
    data[data > maxVal ] = maxVal
    data = data*255 /maxVal
    data = data.astype(np.uint8)
    
    return data

def drawContours(image, contours):
    if( len(np.squeeze(image).shape) ==2): 
        image  = gray2rgb (image)
    
    result_image = label2rgb(contours, image, alpha=0.5)
    return skimage.img_as_ubyte(result_image)

def getEdgeContours(image):
    """
    Applies Otsu Threshold and gets both outer and internal contours. 
    Filters by contour length
    """
    
    image[image>500]=500
    #data = skimage.exposure.equalize_adapthist(image.astype(np.uint16))*255
    local_thresh = skimage.filters.threshold_otsu(image)
    binary = image>local_thresh

    edges = feature.canny(binary, sigma=1)
    all_labels = measure.label(edges)
    
    for label in range(np.max(all_labels)):
        # Edge Length Paramenter should be automated
        if( np.sum(all_labels==label)<100):
            edges[all_labels==label] = 0

    edges = skimage.morphology.thin(edges)
    return edges, binary

def saveTestSamples(data, n=8,  prefix="test", template= None ):
    
    """
    Draws Contours extracted from moving onto fixed samples
    """
    assert n>1
    
    if type(data) == str:
        dataImage = nib.load(data)
        data = dataImage.get_fdata()
    data[data<0] =0
    data = convertToUint8(data)
    
    testSamples = np.arange(0, data.shape[0], data.shape[0]/n)[1:]
    testSamples = testSamples.astype(int)
    
    if template is None:
        for testSample in testSamples:   
            skimage.io.imsave("tests/{}_{}.jpg".format(prefix, testSample),data[testSample] )
        return
    
    if type(template) == str:
        templateImage = nib.load(template)
        template = templateImage.get_fdata()
        template = convertToUint8(template)
        
    for testSample in testSamples:
        e, b = getEdgeContours(template[testSample])
        contouredImage = drawContours(data[testSample] , e)
        
        skimage.io.imsave("tests/{}_{}.jpg".format(prefix, testSample),contouredImage )

## Environment Parameters

In [5]:
alignAxes = True
elastixIter =1

## Load Images

In [6]:
templateImagePath = "average_template_25.nii"
dataImagePath = "B39/brain_25.nii.gz"
annotationImagePath = "annotations/annotation_25.nii.gz"

In [7]:
annotationImage = nib.load(annotationImagePath)
annData = annotationImage.get_fdata()

In [8]:
outputDir = "reg_data2temp_rs"
if not os.path.isdir(outputDir):
    os.mkdir(outputDir)
    
if not os.path.isdir("tests_rs"):
    os.mkdir("tests_rs")

## Axis Alignment

In [9]:
axisAlignedDataPath  = os.path.join(outputDir , "axisAlignedData.nii.gz")

In [10]:
from vol2affine import vol2affine
from align_utils import align_rotation
def getAlignAxisAffineMatrix(fixed, moving):
    
    
    R, r1, r2 = vol2affine(moving=moving, template=fixed,pivot=(0, 0, 0))
    #R = align_rotation(r1,r2)
    origin = np.array(moving.shape)/2

    A1 = np.eye(4)
    A1[0:3, 3] = -origin

    A2 = np.eye(4)
    A2[0:3,0:3] = R[0:3,0:3]

    A3 = np.eye(4)
    A3[0:3, 3] = origin
    A = (A3@A2)@A1
    return A

In [11]:
if alignAxes:
    templateImage = nib.load(templateImagePath)
    template = templateImage.get_fdata()

    dataImage = nib.load(dataImagePath)
    data = dataImage.get_fdata()
    
    A = getAlignAxisAffineMatrix(template, data)
    alignedData = affine_transform(data,np.linalg.inv(A), output_shape = data.shape, order =1)
    alignedData[alignedData<0] =0
    create_nifti_image(alignedData, 2.5, axisAlignedDataPath, 1)
    
    dataImage = nib.load(axisAlignedDataPath)
    data = dataImage.get_fdata()

100%|███████████████████████████████████████████████████████████████████████████████| 528/528 [00:02<00:00, 237.62it/s]


In [12]:
saveTestSamples(dataImagePath, 8, "original")
saveTestSamples(axisAlignedDataPath,8, "axisAligned")

In [13]:
fixedImagePath = templateImagePath


## Elastix

In [14]:
movingImagePath = axisAlignedDataPath

In [17]:
def rescaleMaxTo255(data):
    """
    Thresholds  and converts data to UINT8
    """
    maxVal  = np.percentile(data, 99)
    data[data > maxVal ] = maxVal
    data = data*255 /maxVal
    return data

In [18]:
mImage = nib.load(axisAlignedDataPath)
mdata = mImage.get_fdata()
mdata = rescaleMaxTo255(mdata)

create_nifti_image(mdata, 2.5, os.path.join(outputDir, "mrescaled.nii.gz"),1)

In [19]:
elastixResult  = elastixRegistration(fixedImagePath , os.path.join(outputDir, "mrescaled.nii.gz"), "testrescale")
elastixResult = elastixTransformation(axisAlignedDataPath, "testrescale")

fImage = nib.load(fixedImagePath)
fdata = fImage.get_fdata()
tImage = nib.load(elastixResult)
tdata = tImage.get_fdata()
saveTestSamples(tdata , 8 , "elastix" , templateImagePath)

print("Mutual Information:{}".format(getMutualInformation(fdata, tdata)))

C:\Users\3i\AppData\Local\Temp\ipykernel_6600\4042179872.py:27: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 500 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  image[image>500]=500


Mutual Information:-0.7354833557664661


## Elastix Keyframe + 2D Laplacian

In [20]:
movingImagePath = elastixResult

In [21]:
try:
    deformationField = sliceToSlice2DLaplacian(fixedImagePath , movingImagePath)
    np.save(os.path.join(outputDir,"deformation2d.npy"), deformationField)
    transformedData   = applyDeformationField(movingImagePath , deformationField)
    create_nifti_image(transformedData, 2.5, os.path.join(outputDir, "elastix2DRefined.nii.gz"), 1/2)
    fImage = nib.load(fixedImagePath)
    fdata = fImage.get_fdata()
    print("Mutual Information:{}".format(getMutualInformation(fdata, transformedData)))
    saveTestSamples(transformedData , 8 , "elastix2DRefined" , templateImagePath)
except Exception as e:
    traceback.print_exc()

100%|████████████████████████████████████████████████████████████████████████████████| 528/528 [24:46<00:00,  2.82s/it]


Mutual Information:-0.8110337440755004


C:\Users\3i\AppData\Local\Temp\ipykernel_6600\4042179872.py:27: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 500 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  image[image>500]=500


In [22]:
saveTestSamples(transformedData , 8 , "elastix2DRefined" , templateImagePath)

C:\Users\3i\AppData\Local\Temp\ipykernel_6600\4042179872.py:27: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 500 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  image[image>500]=500


## Elastix + 3D Laplacian 

In [23]:
fixedImagePath = templateImagePath

movingImagePath = elastixResult

In [24]:
try:
    deformationField  = sliceToSlice3DLaplacian(fixedImagePath , movingImagePath , axis =0 )
    np.save(os.path.join(outputDir,"deformation3d.npy"), deformationField)
    transformedData   = applyDeformationField(movingImagePath , deformationField)
    create_nifti_image(transformedData, 2.5, os.path.join(outputDir, "elastixRefined.nii.gz"), 1/2)

    fImage = nib.load(fixedImagePath)
    fdata = fImage.get_fdata()
    print("Mutual Information:{}".format(getMutualInformation(fdata, transformedData)))
    saveTestSamples(transformedData , 8 , "elastix3DRefined" , templateImagePath)
except Exception as e:
    traceback.print_exc()

100%|████████████████████████████████████████████████████████████████████████████████| 528/528 [10:33<00:00,  1.20s/it]


Building data for Laplacian Sparse Matrix A
Creating Laplacian Sparse Matrix A
dx calculated in 239.79392957687378s
dz calculated in 370.6237483024597s
Mutual Information:-0.8194379842870538


C:\Users\3i\AppData\Local\Temp\ipykernel_6600\4042179872.py:27: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 500 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  image[image>500]=500


## 3D - 3D

In [31]:
fixedImagePath =templateImagePath 
movingImagePath = axisAlignedDataPath

In [32]:
try:
    A, deformationField, transformedData = reg3D(fixedImagePath, movingImagePath , 25, 5,75)
    joblib.dump( [A, deformationField], os.path.join(outputDir,"deformationl3d.pkl"))
    #transformedData   = applyDeformationField(movingImagePath , deformationField)
    create_nifti_image(transformedData, 2.5, os.path.join(outputDir, "laplacian3d.nii.gz"), 1/2)
    print("Mutual Information:{}".format(getMutualInformation(fdata, transformedData)))
    saveTestSamples(transformedData , 8 , "laplacian3D" , templateImagePath)
except Exception as e:
    traceback.print_exc()

448039it [01:03, 7070.21it/s]
208293it [00:19, 10829.41it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [01:59<00:00,  4.80s/it]
443830it [01:02, 7146.40it/s]
448039it [00:40, 11144.77it/s]


Building data for Laplacian Sparse Matrix A
Creating Laplacian Sparse Matrix A
dx calculated in 240.38252997398376s
dy calculated in 479.2745804786682s
dz calculated in 681.1365797519684s
Mutual Information:-0.6758897603795798


C:\Users\3i\AppData\Local\Temp\ipykernel_10448\4042179872.py:27: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 500 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  image[image>500]=500


## Area Keyframe + 2D Laplacian

In [25]:
fixedImagePath =templateImagePath 
movingImagePath = axisAlignedDataPath
fImage = nib.load(fixedImagePath)
fdata = fImage.get_fdata()

In [26]:
try:
    A, deformationField, transformedData = areaKeyFrame2DLaplacian(fixedImagePath , movingImagePath, 25, 5,75)
    joblib.dump( [A, deformationField], os.path.join(outputDir,"areaKeyFrameDeformation2d.pkl"))    
    #affineTransformedData = applyAffineTransform(movingImagePath , A,fdata.shape )
    #transformedData   = applyDeformationField(affineTransformedData , deformationField)
    create_nifti_image(transformedData, 2.5, os.path.join(outputDir, "keyframe2DRefined.nii.gz"), 1/2)
    print("Mutual Information:{}".format(getMutualInformation(fdata, transformedData)))
    saveTestSamples(transformedData , 8 , "areaKeyFrame2DLaplacian" , templateImagePath)

except Exception as e:
    traceback.print_exc()

448039it [01:04, 6905.12it/s]
208293it [00:19, 10646.20it/s]
 36%|█████████████████████████████▉                                                     | 9/25 [00:48<01:25,  5.37s/it]
444640it [01:02, 7068.44it/s]


Building data for Laplacian Sparse Matrix A
Creating Laplacian Sparse Matrix A


  0%|                                                                                          | 0/528 [00:00<?, ?it/s]C:\Users\3i\anaconda3\envs\tissuecyte\lib\site-packages\numpy\lib\function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
100%|████████████████████████████████████████████████████████████████████████████████| 528/528 [28:23<00:00,  3.23s/it]


Mutual Information:-0.6394556848654188


C:\Users\3i\AppData\Local\Temp\ipykernel_10448\4042179872.py:27: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 500 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  image[image>500]=500


## Area Keyframe + 3D Laplacian

In [27]:
fixedImagePath =templateImagePath 
movingImagePath = axisAlignedDataPath
fImage = nib.load(fixedImagePath)
fdata = fImage.get_fdata()

In [28]:
try:
    A, deformationField,transformedData = areaKeyFrame3DLaplacian(fixedImagePath , movingImagePath, 25,5, 75)
    joblib.dump( [A, deformationField], os.path.join(outputDir,"areaKeyFrameDeformation3d.pkl"))    
    #np.save("areaKeyFrameDeformation3d.npy", np.array([A, deformationField]))
    #affineTransformedData = applyAffineTransform(movingImagePath , A,fdata.shape )
    #transformedData   = applyDeformationField(affineTransformedData , deformationField)
    create_nifti_image(transformedData, 2.5, os.path.join(outputDir, "keyframe3DRefined.nii.gz"), 1/2)
    print("Mutual Information:{}".format(getMutualInformation(fdata, transformedData)))    
    saveTestSamples(transformedData , 8 , "areaKeyFrame3DLaplacian" , templateImagePath)
except Exception as e:
    traceback.print_exc()

448039it [01:04, 6978.01it/s]
208293it [00:19, 10435.76it/s]
 36%|█████████████████████████████▉                                                     | 9/25 [00:48<01:26,  5.42s/it]
444822it [01:02, 7143.15it/s]


Building data for Laplacian Sparse Matrix A
Creating Laplacian Sparse Matrix A


100%|████████████████████████████████████████████████████████████████████████████████| 528/528 [10:58<00:00,  1.25s/it]


Building data for Laplacian Sparse Matrix A
Creating Laplacian Sparse Matrix A
dx calculated in 297.7416498661041s
dz calculated in 581.210691690445s
Mutual Information:-0.6551787317562096


C:\Users\3i\AppData\Local\Temp\ipykernel_10448\4042179872.py:27: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 500 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  image[image>500]=500


In [29]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import nibabel as nib
import SimpleITK as sitk
import napari

import plotly.express as px
import plotly.graph_objects as go

from scipy import ndimage
from scipy.ndimage import gaussian_filter
import open3d as o3d

from skimage.util import random_noise
from skimage import feature
from skimage import filters
from skimage import measure
from scipy.ndimage import affine_transform
from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.filters import sobel
from skimage.exposure import match_histograms

from scipy.sparse import csr_matrix, linalg as sla

from registration import *
from plot_utils import *
from edge_utils import *


import open3d
import os
import gc
#from mayavi import mlab

import scipy
import pyamg

In [30]:
def getContours(sno, fdata, mdata, axis=0):
    
    fpercentile  = np.percentile(fdata, 99)
    fdata[fdata > fpercentile ] = fpercentile
    
    fixedimage = np.take(fdata, sno, axis=axis)
    movingimage = np.take(mdata, sno, axis=axis)
    
    fedge , fbin = getEdgeContours(fixedimage)
    medge , mbin = getEdgeContours(movingimage)

    return fedge, medge, fbin, mbin